# Why python for data analysis, machine learning?
There are lots of reasons that we want to use python for doing data science. It is certainly one of the younger programming languages used in the data science ecosystem (compared to say R and SAS) but it is used just as frequently for analysis as SAS and R. Having a good foundation in python and R, (and SAS or SPSS) should be a *must* for **every data scientist** and machine learning enthusiast. 

In this course, python allows for an open source method of performing machine learning that runs from just about any machine. So let's start with looking at Numpy and Pandas pachages for analyzing data. 

With that in mind, let's go over the following:
- Numpy matrices
- Simple operations on arrays and matrices
- Indexing with numpy
- Pandas for tabular data
- Representing categorical data (discussion point)

In [1]:
import sys
import numpy as np

print(sys.version)
print(np.__version__)

3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
1.19.2


In [2]:
x = np.random.rand(5,3)
x

array([[0.52635617, 0.45349981, 0.71400252],
       [0.36066163, 0.59249041, 0.0149725 ],
       [0.58542855, 0.94053073, 0.65781546],
       [0.05268799, 0.83211198, 0.11682865],
       [0.9348768 , 0.0474619 , 0.23551532]])

In [3]:
x.shape

(5, 3)

In [4]:
x.dtype

dtype('float64')

In [5]:
y = np.random.rand(3,4)
z = x*y
z

ValueError: operands could not be broadcast together with shapes (5,3) (3,4) 

In [6]:
# we can designate what matrix multiplication is directly using objects
z = np.dot(x,y)
z

array([[0.85561243, 0.73675757, 1.19504333, 1.05634327],
       [0.70620757, 0.35074942, 0.37320316, 0.55819018],
       [1.19386783, 0.96560097, 1.21535494, 1.33247437],
       [0.55015363, 0.52480128, 0.19497293, 0.59166092],
       [0.97853372, 0.23764787, 1.09705306, 0.7232309 ]])

In [7]:
# or we can use the overloaded matrix multiplication operator
z = x @ y
z

array([[0.85561243, 0.73675757, 1.19504333, 1.05634327],
       [0.70620757, 0.35074942, 0.37320316, 0.55819018],
       [1.19386783, 0.96560097, 1.21535494, 1.33247437],
       [0.55015363, 0.52480128, 0.19497293, 0.59166092],
       [0.97853372, 0.23764787, 1.09705306, 0.7232309 ]])

# Indexing

In [8]:
x1 = np.array([[1,2,3],
               [4,5,6],
               [7,8,9]])
x1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [9]:
for row in range(x1.shape[0]):
    print(x1[row,1])

2
5
8


In [10]:
print(x1[:,1])
print(x1[:,1]>3)
# slicing
print(x1[ x1[:,1]>3 ])

[2 5 8]
[False  True  True]
[[4 5 6]
 [7 8 9]]


In [11]:
x2 = np.array(range(10))
print(x2)
x2.shape

[0 1 2 3 4 5 6 7 8 9]


(10,)

In [12]:
idx = x2>5
print(idx)
print(x2[idx])

[False False False False False False  True  True  True  True]
[6 7 8 9]


In [13]:
x2[x2>5]

array([6, 7, 8, 9])

# Named columns
So what if we have a matrix of data where each row is some observation of features and the feature values are represented in each column?

In [14]:
col_names = ['temperature','time','day']
data = np.array([[64,2100,1],
                 [50,2200,4],
                 [48,2300,3],
                 [34,0,   2],
                 [30,100, 5]])
data

array([[  64, 2100,    1],
       [  50, 2200,    4],
       [  48, 2300,    3],
       [  34,    0,    2],
       [  30,  100,    5]])

In [15]:
data2 = data[data[:,1]>1500]
data2

array([[  64, 2100,    1],
       [  50, 2200,    4],
       [  48, 2300,    3]])

In [16]:
# pandas to the rescue
import pandas as pd

df = pd.DataFrame(data,columns=col_names)
df

,temperature,time,day
0,64,2100,1
1,50,2200,4
2,48,2300,3
3,34,0,2
4,30,100,5


In [17]:
# can always access the backend numpy with .values
print(type(df.to_numpy()))
df.to_numpy()

<class 'numpy.ndarray'>


array([[  64, 2100,    1],
       [  50, 2200,    4],
       [  48, 2300,    3],
       [  34,    0,    2],
       [  30,  100,    5]])

In [18]:
df[df.time>1500]

,temperature,time,day
0,64,2100,1
1,50,2200,4
2,48,2300,3


In [19]:
# lets get a description of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   temperature  5 non-null      int64
 1   time         5 non-null      int64
 2   day          5 non-null      int64
dtypes: int64(3)
memory usage: 248.0 bytes


In [20]:
df.day[df.day==1] = 'Mon'
df

,temperature,time,day
0,64,2100,Mon
1,50,2200,4
2,48,2300,3
3,34,0,2
4,30,100,5


In [21]:
# there is almost always a more efficient built in pandas function
df.day.replace(to_replace=range(7),
               value=['Su','Mon','Tues','Wed','Th','Fri','Sat'],
               inplace=True)
df

,temperature,time,day
0,64,2100,Mon
1,50,2200,Th
2,48,2300,Wed
3,34,0,Tues
4,30,100,Fri


In [22]:
# notice how the type of the column has changed to an object "categorical"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   temperature  5 non-null      int64 
 1   time         5 non-null      int64 
 2   day          5 non-null      object
dtypes: int64(2), object(1)
memory usage: 248.0+ bytes


In [23]:
# one hot encoding example
pd.get_dummies(df.day)

,Fri,Mon,Th,Tues,Wed
0,0,1,0,0,0
1,0,0,1,0,0
2,0,0,0,0,1
3,0,0,0,1,0
4,1,0,0,0,0


# Some Pandas Syntax

In [24]:
print(df.day)
print(df['day'])
df[['day','temperature']]

0     Mon
1      Th
2     Wed
3    Tues
4     Fri
Name: day, dtype: object
0     Mon
1      Th
2     Wed
3    Tues
4     Fri
Name: day, dtype: object


,day,temperature
0,Mon,64
1,Th,50
2,Wed,48
3,Tues,34
4,Fri,30


In [25]:
print(df.day[2])
print(df.day[2:])

Wed
2     Wed
3    Tues
4     Fri
Name: day, dtype: object


In [26]:
df.iloc[3:]

,temperature,time,day
3,34,0,Tues
4,30,100,Fri


In [27]:
df.iloc[3:][['day','temperature']]

,day,temperature
3,Tues,34
4,Fri,30


In [28]:
df[['day','temperature']].iloc[3:]

,day,temperature
3,Tues,34
4,Fri,30


In [29]:
df.mean()

temperature      45.2
time           1340.0
dtype: float64

In [30]:
df.std()

temperature      13.608821
time           1180.254210
dtype: float64

In [31]:
df.mean()/df.std()

temperature    3.321375
time           1.135349
dtype: float64

In [32]:
df.time.unique()

array([2100, 2200, 2300,    0,  100])

# Pandas Block Manager
Let's take a look at some important points from the following post:
 - https://uwekorn.com/2020/05/24/the-one-pandas-internal.html

This is the pandas BlockManager, which tries to group internal structures together to make things fast:
<img src="https://uwekorn.com/images/pd-df-perception.002.png" width=200 height=200 />

In [33]:
df

,temperature,time,day
0,64,2100,Mon
1,50,2200,Th
2,48,2300,Wed
3,34,0,Tues
4,30,100,Fri


In [34]:
print(df._data.nblocks)
df._data

2


BlockManager
Items: Index(['temperature', 'time', 'day'], dtype='object')
Axis 1: RangeIndex(start=0, stop=5, step=1)
IntBlock: slice(0, 2, 1), 2 x 5, dtype: int64
ObjectBlock: slice(2, 3, 1), 1 x 5, dtype: object

## Advantages and disadvantages:
This can speed up operations because it inhenertly can apply operations along columns in a single pass over the data (like sums, etc.) and therefore is using c++ for much of the heavy lifting.

But, **it might be bad** when you are adding columns to the data because it can trigger consolidation of columns, which means copying over data in numpy to creata new matrix. The slow down also doesn't show up until a needed column is accessed (lazy data copying). Let's do an example from:  https://uwekorn.com/2020/05/24/the-one-pandas-internal.html

**Block consolidation is triggered after 100 blocks of data are reached.**

In [35]:
df_example = pd.DataFrame({
    'int64': np.arange(1024 * 1024, dtype=np.int64),
    'float64': np.arange(1024 * 1024, dtype=np.float64),
})
df_example

,int64,float64
0,0,0.0
1,1,1.0
2,2,2.0
3,3,3.0
4,4,4.0
...,...,...
1048571,1048571,1048571.0
1048572,1048572,1048572.0
1048573,1048573,1048573.0
1048574,1048574,1048574.0


In [36]:
%%time 
for i in range(97):
    df_example[f'new_{i}'] = df_example['int64'].values
    
print(df_example._data.nblocks)
df_example

99
CPU times: user 305 ms, sys: 229 ms, total: 534 ms
Wall time: 533 ms


,int64,float64,new_0,new_1,new_2,new_3,new_4,new_5,new_6,new_7,...,new_87,new_88,new_89,new_90,new_91,new_92,new_93,new_94,new_95,new_96
0,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1.0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,2.0,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3.0,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4.0,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048571,1048571,1048571.0,1048571,1048571,1048571,1048571,1048571,1048571,1048571,1048571,...,1048571,1048571,1048571,1048571,1048571,1048571,1048571,1048571,1048571,1048571
1048572,1048572,1048572.0,1048572,1048572,1048572,1048572,1048572,1048572,1048572,1048572,...,1048572,1048572,1048572,1048572,1048572,1048572,1048572,1048572,1048572,1048572
1048573,1048573,1048573.0,1048573,1048573,1048573,1048573,1048573,1048573,1048573,1048573,...,1048573,1048573,1048573,1048573,1048573,1048573,1048573,1048573,1048573,1048573
1048574,1048574,1048574.0,1048574,1048574,1048574,1048574,1048574,1048574,1048574,1048574,...,1048574,1048574,1048574,1048574,1048574,1048574,1048574,1048574,1048574,1048574


In [37]:
%time df_example['dummy_name3'] = df_example['int64'].values # copy over some new columns
print('Number of blocks in data:',df_example._data.nblocks)

%time df_example['dummy_name4'] = df_example['int64'].values # copy over some new columns
print('Number of blocks in data:',df_example._data.nblocks)


CPU times: user 7.27 ms, sys: 9.34 ms, total: 16.6 ms
Wall time: 12.3 ms
Number of blocks in data: 100
CPU times: user 518 ms, sys: 772 ms, total: 1.29 s
Wall time: 1.29 s
Number of blocks in data: 2
